In [5]:
from problems.HS100 import HS100, CallableClass
import methods.methods as methods
from typing import Tuple, Union, List, Type
from copy import deepcopy
import pandas as pd
import numpy as np
from numpy import number
from scipy.stats.qmc import LatinHypercube as lhs
from scipy.stats.qmc import scale
import scipy.stats as stats
from smt.surrogate_models import KRG
import random

In [6]:
def sample_sites(problem: dict, n_samples: int, seed=random.randint(1,1000)) -> pd.DataFrame:
    """ Simple wrapper to sample sites
    
    Parameters
    ----------
    problem : dict
        The variables and constraints given

    n_sample: int
        The number of sites to generare

    seed: int
        Seed for the LHS sampling
   
    Returns
    -------
    pd.DataFrame
        The DataFrame of sites
    """

    variables = list(problem["variables"].keys())
    nind = len(variables)
    # Get all the bounds for the variables
    bounds = np.array([problem["variables"][var]["bounds"] for var in variables])
    # Generate the experiment 
    lhs_instance = lhs(
        nind,
        scramble=True,
        strength=1,
        optimization=None,
        seed=seed,
    )
    # Get the experiment
    normalized_array = lhs_instance.random(n_samples)
    # Scale using the bounds
    exp_array = scale(normalized_array, bounds[:, 0], bounds[:, 1])
    # Create the DataFrame
    exp_df = pd.DataFrame(data=exp_array, columns=variables)
    return(exp_df)


In [7]:
def evaluate_sites(local_eval: CallableClass, exp_data: pd.DataFrame, verbose=True):
    """ evaluates sites passed and returns their constraint violation data

        Parameters
        ----------
        local_eval : CallableClass
            The example evaluator to use
        
        n_sample: int
            The number of sites to generare
   
        Returns
        -------
        pd.DataFrame
            The DataFrame of evaluated experimental sites with constraint violation
    """
    eps = 1e-6
    local_problem = local_eval.problem()
    local_eval(exp_data)
    my_constraint_calculator = methods.ConstraintCalculator(local_problem)
    exp_data['__conviol__'] = my_constraint_calculator(exp_data)
    exp_data['__State__'] = pd.cut(exp_data['__conviol__'], [-np.inf, eps, 100*eps, np.inf], include_lowest=True, labels=['Feasible', 'Nearly Feasible', "Infeasible"]).astype("str")
    feasible_sites = (exp_data['__State__'] == 'Feasible').sum()
    percentage = feasible_sites/num_sites * 100
    if verbose: 
        print(f"Test evaluator {local_eval.name} with {len(local_problem['variables'])} variables {percentage}% feasible sites")
    return (exp_data)

In [8]:
def advanced_testing(local_eval: CallableClass, num_sites_training: int, num_sites_testing : int, verbose=True) -> pd.DataFrame:
    """ Wrapper to create an experiment, evaluate the passed in function, create a surrogate model, evaluate model

        Parameters
        ----------
        local_eval : CallableClass
            The example evaluator to use
        
        n_sample_training: int
            The number of sites to train the sm on

        n_sample_testing: int
            The number of sites to test the sm
   
        Returns
        -------
        pd.DataFrame
            The DataFrame of evaluated experimental sites with constraint violation
    """
    # get training data
    sample_data = sample_sites(local_eval.problem(), num_sites_training)
    exp_data = evaluate_sites(local_eval, sample_data, verbose=False)
    variables = list(local_eval.problem()["variables"].keys())
    xt = exp_data[variables].to_numpy()
    yt = exp_data['__conviol__'].to_numpy()

    # create model
    sm = KRG(theta0=[1e-2])
    sm.set_training_values(xt, yt)
    sm.train()

    # get testing data
    eps = 1e-6
    x = sample_sites(local_eval.problem(), num_sites_testing).to_numpy()
    y = sm.predict_values(x).flatten()
    feasible_points = pd.DataFrame(data=x[y < eps], columns=variables)
    
    # evaluate model on testing data
    feasible_points = evaluate_sites(local_eval, feasible_points, verbose=verbose)
    return (feasible_points) 
    

In [15]:
hs100 = HS100()
problem = hs100.problem()
nind = len(problem['variables'])
num_sites = 50
test_data = advanced_testing(hs100, num_sites, num_sites**2)
test_data

___________________________________________________________________________
   
                                  Kriging
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 50
   
___________________________________________________________________________
   
 Training
   
   Training ...
   Training - done. Time (sec):  1.8788700
___________________________________________________________________________
   
 Evaluation
   
      # eval points. : 2500
   
   Predicting ...
   Predicting - done. Time (sec):  0.0458748
   
   Prediction time/pt. (sec) :  0.0000183
   
Test evaluator hs100 with 7 variables 12.0% feasible sites


,x1,x2,x3,x4,x5,x6,x7,f,c1,c2,c3,c4,__conviol__,__State__
0,9.513716,0.709838,1.795151,-3.534031,3.659800,-5.305131,9.808588,3.494693e+04,-124.834902,188.242622,-113.717159,-214.312511,2.429347,Infeasible
1,-2.033564,-1.815720,-1.426314,-7.692207,-1.219600,1.200575,1.041103,2.170350e+03,-143.034069,287.810986,239.155678,-7.380712,0.160954,Infeasible
2,1.076528,-3.373826,1.299561,-2.719441,-2.351217,0.172060,7.228577,6.184403e+03,-283.140241,268.065406,217.508139,48.361924,0.283140,Infeasible
3,-1.100670,1.976727,-1.623781,8.945398,8.155501,5.980613,-0.358659,2.943263e+06,-280.461688,256.617965,-0.067705,-54.402156,0.612061,Infeasible
4,6.782806,3.542999,-0.079924,1.982043,1.085039,1.270923,8.941013,6.900537e+03,-458.793570,222.930480,89.279264,-32.500486,0.562245,Infeasible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,6.315740,-2.360160,1.391373,-0.830484,9.858879,5.233374,6.577641,9.185878e+06,-99.308295,236.200474,-66.540453,-167.527788,1.805320,Infeasible
94,7.383725,1.246314,1.627915,5.107454,0.223226,-8.245299,7.775853,5.104724e+03,-96.365377,195.189674,-321.081852,-70.562903,3.288853,Infeasible
95,1.279555,-2.987017,-1.286912,2.397080,0.988080,-4.047649,7.114874,4.209144e+03,-141.732769,264.033738,116.266168,68.252105,0.141733,Infeasible
96,-2.345984,-2.220591,-2.034657,6.387881,1.877907,-8.695820,5.764200,3.557709e+03,-127.527221,259.175385,-162.563518,87.288468,1.630630,Infeasible


## Experiment 3
1. Generate training data once and train
2. Regenerate sites until enough pass the model's filter
3. Train again, and repeat

In [10]:
# multi-run training
def experiment_3(local_eval, sites_per_run, runs):
    problem = hs100.problem()
    nind = len(problem['variables'])

    # create model
    sm = KRG(theta0=[1e-2], print_prediction=False)
    
    for i in range(runs):
        if i == 0:
            # get first run's data
            sample_data = sample_sites(local_eval.problem(), sites_per_run)
            exp_data = evaluate_sites(local_eval, sample_data, verbose=False)
        variables = list(local_eval.problem()["variables"].keys())
        xt = exp_data[variables].to_numpy()
        yt = exp_data['__conviol__'].to_numpy()
        
        # train model
        sm.set_training_values(xt, yt)
        sm.train()
    
        # get next run's data
        eps = 1e-6
        x = sample_sites(local_eval.problem(), sites_per_run**2).to_numpy()
        y = sm.predict_values(x).flatten()
        while len(y[y < eps]) < sites_per_run:
            # continue generating sites until enough
            x_p = sample_sites(local_eval.problem(), sites_per_run**2).to_numpy()
            y_p = data=sm.predict_values(x_p).flatten()
            x = np.concatenate((x, x_p))
            y = np.concatenate((y, y_p))
        exp_data = pd.DataFrame(data=x[y < eps], columns=variables)
        
        # evaluate model on testing data
        exp_data = evaluate_sites(local_eval, exp_data)
    return exp_data

In [11]:
hs100 = HS100()
problem = hs100.problem()
nind = len(problem['variables'])
num_sites = 10
#experiment_3(hs100, num_sites, 3)

## Experiment 4
1. Train to maximize $-\log(C+\epsilon)$
2. Filter is now if EI(x) is sufficiently large,
    $EI(x) = (f^*-\mu_f(x))\Phi\left(\frac{f^*-\mu_f(x)}{\sigma_f(x)}\right)+\sigma_f(x)\phi\left(\frac{f^*-\mu_f(x)}{\sigma_f(x)}\right)$
   where $\mu_f,\sigma_f, f^*$ are the predicted value, variance, and maximum of the sm, and $\Phi,\phi$ are normal CDF and PDF.
   
   Note: Need to choose test sites wisely and tune filter better

In [12]:
# weigh with variance
def experiment_4(local_eval, num_sites_training, num_sites_testing,verbose=True):
    # get training data
    sample_data = sample_sites(local_eval.problem(), num_sites_training)
    exp_data = evaluate_sites(local_eval, sample_data, verbose=False)
    variables = list(local_eval.problem()["variables"].keys())
    xt = exp_data[variables].to_numpy()
    yt = (-1)*np.log(exp_data['__conviol__'].to_numpy() + 1e-6)
    
    # create model
    sm = KRG(theta0=[1e-2], print_global=False)
    sm.set_training_values(xt, yt)
    sm.train()

    # compute EI(x)
    x = sample_sites(local_eval.problem(), num_sites_testing).to_numpy()
    mu = sm.predict_values(x).flatten()
    sigma = np.sqrt(sm.predict_variances(x).flatten())
    y_max = np.max(mu)
    t = (y_max-mu)/sigma
    EI_x = (y_max-mu)*stats.norm.cdf(t) + sigma*stats.norm.pdf(t)
    feasible_points = pd.DataFrame(data=x[EI_x > 5], columns=variables)
    
    # evaluate model on testing data
    feasible_points = evaluate_sites(local_eval, feasible_points, verbose=verbose)
    return (feasible_points) 

In [13]:
hs100 = HS100()
problem = hs100.problem()
nind = len(problem['variables'])
num_sites = 25
experiment_4(hs100, num_sites, num_sites**2)

Test evaluator hs100 with 7 variables 8.0% feasible sites


,x1,x2,x3,x4,x5,x6,x7,f,c1,c2,c3,c4,__conviol__,__State__
0,-5.529141,-5.758379,-8.743796,-2.246104,3.816640,5.953620,2.898170,3.926565e+04,-3263.206691,-420.497861,100.523115,-210.724030,5.724581,Infeasible
1,9.818763,-3.194266,4.946372,-9.689501,-1.825226,-2.381453,-0.948382,3.469760e+03,-749.506437,-13.950216,-81.649846,-537.385115,5.488731,Infeasible
2,-5.104010,9.435533,-3.590033,2.458193,4.977514,9.688073,1.711252,1.532157e+05,-23749.207818,163.057430,-325.099596,-393.103360,24.290880,Infeasible
3,4.778466,-5.511930,6.538125,0.574757,-3.733520,8.169195,-4.962849,3.199183e+04,-2676.942623,-166.692543,-384.403321,-381.663500,6.268011,Infeasible
4,-7.079450,5.410729,0.845053,-5.885154,-8.109177,7.503834,-0.415930,2.845249e+06,-2643.326158,305.958802,-11.621191,-388.188020,4.697837,Infeasible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,8.523849,5.921021,5.540704,-2.711425,8.459330,-6.378782,4.994850,3.667261e+06,-3782.848782,-91.253210,-239.281401,-148.834381,4.804517,Infeasible
606,5.711529,4.348007,-4.848890,-4.507684,2.145181,-7.233120,-5.766094,3.983204e+03,-1097.612667,0.510823,-314.307210,-149.175007,3.648147,Infeasible
607,-9.345215,-7.620601,-1.615610,9.508069,7.776385,5.154177,4.755073,2.214218e+06,-10564.159378,342.444659,231.513743,-172.442722,10.703976,Infeasible
608,-4.996386,6.993529,0.753814,4.765775,-2.972873,-9.486564,-2.773710,8.071411e+03,-7176.068861,282.573102,-300.151593,-237.806477,8.133895,Infeasible
